In [1]:
# Cell 1: imports and settings
import os, random, time, re
import pandas as pd, numpy as np, torch
from sklearn.metrics import accuracy_score, f1_score, classification_report

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset as HFDataset

from textattack import Attacker, AttackArgs
from textattack.attack_recipes import TextFoolerJin2019, DeepWordBugGao2018, BAEGarg2019
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import Dataset as TA_Dataset

from textblob import TextBlob

# user-configurable
DATA_PATH = "labeled_news_comments.csv"   # <- 改成你的文件路径
TEST_SAMPLE_SIZE = 1000
ATTACK_SAMPLE_SIZE = 200
ADV_GEN_PER_ATTACK = 200
ADV_TRAIN_EPOCHS = 2
USE_EXISTING_MODEL = False
MODEL_PATH = "./best_model"   # 用已有模型时设置为 True 并改路径
RANDOM_SEED = 42

# reproducible
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Device: cuda


In [2]:
# Cell 2: load and sample data
df = pd.read_csv(DATA_PATH)
# basic cleaning
df['body'] = df['body'].astype(str).apply(lambda x: re.sub(r'http\\S+|www\\.\\S+', '', x))
df['body'] = df['body'].apply(lambda x: re.sub(r'\\s+', ' ', x).strip())
df = df.dropna(subset=['body','label']).reset_index(drop=True)
df['label'] = df['label'].astype(int)

print("Total rows:", len(df))

# sample test set and remaining for training subset
test_df = df.sample(min(TEST_SAMPLE_SIZE, len(df)), random_state=RANDOM_SEED).reset_index(drop=True)
remaining = df.drop(test_df.index, errors='ignore').reset_index(drop=True)
# base train subset (for quick demo; can enlarge)
train_subset = remaining.sample(min(2000, len(remaining)), random_state=RANDOM_SEED).reset_index(drop=True)

print("test_df size:", len(test_df), "train_subset size:", len(train_subset))


Total rows: 149659
test_df size: 1000 train_subset size: 2000


In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
# Load or quick-fine-tune a model (if you have a saved model set USE_EXISTING_MODEL = True)
USE_EXISTING_MODEL = True
model_name = 'bert-base-uncased'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

if USE_EXISTING_MODEL:
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
    model.load_state_dict(torch.load("best_model.pt"))
    print('Model loaded')
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # quick finetune on a small subset (demo only)
    small_train = train_subset.sample(min(800, len(train_subset)), random_state=RANDOM_SEED)
    small_val = train_subset.sample(min(200, len(train_subset)), random_state=RANDOM_SEED+1)

    def tokenize_fn(ex):
        return tokenizer(ex['body'], truncation=True, padding='max_length', max_length=128)

    hf_train = HFDataset.from_pandas(small_train).map(tokenize_fn, batched=True)
    hf_val = HFDataset.from_pandas(small_val).map(tokenize_fn, batched=True)
    hf_train = hf_train.rename_column('label', 'labels')
    hf_val = hf_val.rename_column('label', 'labels')
    hf_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    hf_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(df['label'].unique())).to(device)

    training_args = TrainingArguments(
        output_dir='./tmp_ft',
        per_device_train_batch_size=8,
        per_device_eval_batch_size=64,
        num_train_epochs=1,
        evaluation_strategy='no',
        save_strategy='no',
        learning_rate=2e-5,
        weight_decay=0.01,
        fp16=torch.cuda.is_available(),
    )

    trainer = Trainer(model=model, args=training_args, train_dataset=hf_train, eval_dataset=hf_val, tokenizer=tokenizer)
    trainer.train()
    model.save_pretrained('./quick_ft_model')
    tokenizer.save_pretrained('./quick_ft_model')
    model = AutoModelForSequenceClassification.from_pretrained('./quick_ft_model').to(device)
    print("Quick-finetuned base model ready")


device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded


In [5]:
# Cell 4: evaluation helper + baseline
def evaluate_texts(model, tokenizer, texts, labels, batch_size=64):
    model.eval()
    preds = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            enc = tokenizer(batch_texts, truncation=True, padding=True, return_tensors='pt').to(model.device)
            logits = model(**enc).logits
            batch_preds = logits.argmax(dim=-1).cpu().numpy().tolist()
            preds.extend(batch_preds)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    return acc, f1, preds

clean_acc, clean_f1, _ = evaluate_texts(model, tokenizer, test_df['body'].tolist(), test_df['label'].tolist(), batch_size=64)
print(f"Clean set -> acc: {clean_acc:.4f}, f1: {clean_f1:.4f}")


D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Clean set -> acc: 0.9860, f1: 0.9861


In [6]:
# Cell 5: build attacks and prepare TA dataset
wrapper = HuggingFaceModelWrapper(model, tokenizer)
attacks = {
    'TextFooler': TextFoolerJin2019.build(wrapper),
    'DeepWordBug': DeepWordBugGao2018.build(wrapper),
    'BAE': BAEGarg2019.build(wrapper),
}
attack_subset = test_df.sample(min(len(test_df), ATTACK_SAMPLE_SIZE), random_state=RANDOM_SEED).reset_index(drop=True)
ta_dataset = TA_Dataset([(row['body'], int(row['label'])) for _, row in attack_subset.iterrows()])
print("attack_subset size =", len(attack_subset))


textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializi

attack_subset size = 200


In [7]:
# Cell 6: run attacks (collect perturbed texts and stats)
attack_outputs = {}         # {attack_name: list of examples}
attack_adv_examples = {}    # {attack_name: adv_examples for augmentation}

for name, attack in attacks.items():
    print("\nRunning attack:", name)
    attack_args = AttackArgs(num_examples=len(ta_dataset), disable_stdout=True, log_to_csv=None)
    attacker = Attacker(attack, ta_dataset, attack_args=attack_args)
    results = attacker.attack_dataset()  # list of AttackResult
    examples = []
    adv_for_aug = []
    for r in results:
        orig_text = r.original_text()
        pert_text = r.perturbed_text() or orig_text
        orig_label = int(r.original_result.ground_truth_output)
        pred_orig = int(r.original_result.output)
        pred_pert = int(r.perturbed_result.output)
        success = (pred_orig != pred_pert)
        examples.append({
            'orig_text': orig_text,
            'pert_text': pert_text,
            'label': orig_label,
            'pred_orig': pred_orig,
            'pred_pert': pred_pert,
            'success': success,
        })
        # collect successful adv examples up to quota
        if success and len(adv_for_aug) < ADV_GEN_PER_ATTACK:
            adv_for_aug.append({'body': pert_text, 'label': orig_label})
    attack_outputs[name] = examples
    attack_adv_examples[name] = adv_for_aug
    print(f"Finished {name}. successful adv collected for aug: {len(adv_for_aug)} / {len(examples)}")



Running attack: TextFooler
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['pre

  0%|          | 0/200 [00:00<?, ?it/s]D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


From D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\tensorflow_hub\resolver.py:120: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.



From D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\tensorflow_hub\module_v2.py:126: The name tf.saved_model.load_v2 is deprecated. Please use tf.compat.v2.saved_model.load instead.

[Succeeded / Failed / Skipped / Total] 149 / 48 / 3 / 200: 100%|██████████| 200/200 [04:54<00:00,  1.47s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 149    |
| Number of failed attacks:     | 48     |
| Number of skipped attacks:    | 3      |
| Original accuracy:            | 98.5%  |
| Accuracy under attack:        | 24.0%  |
| Attack success rate:          | 75.63% |
| Average perturbed word %:     | 17.54% |
| Average num. words per input: | 20.7   |
| Avg num queries:              | 79.96  |
+-------------------------------+--------+

...

[Succeeded / Failed / Skipped / Total] 143 / 54 / 3 / 200: 100%|██████████| 200/200 [01:21<00:00,  2.47it/s]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 143    |
| Number of failed attacks:     | 54     |
| Number of skipped attacks:    | 3      |
| Original accuracy:            | 98.5%  |
| Accuracy under attack:        | 27.0%  |
| Attack success rate:          | 72.59% |
| Average perturbed word %:     | 16.17% |
| Average num. words per input: | 20.7   |
| Avg num queries:              | 22.36  |
+-------------------------------+--------+


Finished DeepWordBug. successful adv collected for aug: 143 / 200

Running attack: BAE
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 99 / 98 / 3 / 200: 100%|██████████| 200/200 [05:20<00:00,  1.60s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 99     |
| Number of failed attacks:     | 98     |
| Number of skipped attacks:    | 3      |
| Original accuracy:            | 98.5%  |
| Accuracy under attack:        | 49.0%  |
| Attack success rate:          | 50.25% |
| Average perturbed word %:     | 18.64% |
| Average num. words per input: | 20.7   |
| Avg num queries:              | 58.96  |
+-------------------------------+--------+
Finished BAE. successful adv collected for aug: 99 / 200


In [8]:
# Cell 7: summarize attack metrics
rows = []
for name, examples in attack_outputs.items():
    y_true = [e['label'] for e in examples]
    y_clean = [e['pred_orig'] for e in examples]
    y_adv = [e['pred_pert'] for e in examples]
    clean_f1 = f1_score(y_true, y_clean, average='weighted')
    adv_f1 = f1_score(y_true, y_adv, average='weighted')
    attack_success_rate = sum(e['success'] for e in examples) / len(examples) if len(examples)>0 else 0.0
    rows.append({'attack': name, 'clean_f1': clean_f1, 'adv_f1': adv_f1, 'f1_drop': clean_f1-adv_f1, 'attack_success_rate': attack_success_rate})
pd.DataFrame(rows)


,attack,clean_f1,adv_f1,f1_drop,attack_success_rate
0,TextFooler,0.985083,0.319355,0.665728,0.745
1,DeepWordBug,0.985083,0.347867,0.637216,0.715
2,BAE,0.985083,0.550991,0.434092,0.495


In [9]:
# Cell 8: spell-correction defense evaluation
def simple_spell_correct(text):
    try:
        return str(TextBlob(text).correct())
    except Exception:
        return text

spell_results = {}
for name, examples in attack_outputs.items():
    labels = []
    corrected_preds = []
    for e in examples:
        corrected = simple_spell_correct(e['pert_text'])
        labels.append(e['label'])
        _, _, preds = evaluate_texts(model, tokenizer, [corrected], [e['label']])
        corrected_preds.append(preds[0])
    spell_f1 = f1_score(labels, corrected_preds, average='weighted') if len(labels)>0 else None
    spell_results[name] = spell_f1
    print(f"{name}: spell-correct F1 = {spell_f1:.4f}" if spell_f1 is not None else f"{name}: no examples")


D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will b

TextFooler: spell-correct F1 = 0.6514


D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will b

DeepWordBug: spell-correct F1 = 0.8988


D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will b

BAE: spell-correct F1 = 0.6158


D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will b

In [10]:
# Cell 9: mixed adversarial training
# collect adv examples across attacks
mixed_adv_examples = []
for name, advs in attack_adv_examples.items():
    mixed_adv_examples.extend(advs)
print("collected adv examples total:", len(mixed_adv_examples))

# if too few adv examples, fallback to duplicating train_subset
if len(mixed_adv_examples) < 50:
    print("Not enough adv examples; fallback to using train_subset duplicate augmentation")
    aug_df = pd.concat([train_subset, train_subset.sample(min(200, len(train_subset)), random_state=RANDOM_SEED)]).reset_index(drop=True)
else:
    adv_df = pd.DataFrame(mixed_adv_examples)
    aug_df = pd.concat([train_subset, adv_df]).reset_index(drop=True)

print("Augmented train size:", len(aug_df))

# prepare HF dataset
hf_aug = HFDataset.from_pandas(aug_df).map(lambda ex: tokenizer(ex['body'], truncation=True, padding='max_length', max_length=128), batched=True)
hf_aug = hf_aug.rename_column('label', 'labels')
hf_aug.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# fine-tune adv_model
adv_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['label'].unique())).to(device)
adv_training_args = TrainingArguments(
    output_dir='./adv_mixed_ft',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    num_train_epochs=ADV_TRAIN_EPOCHS,
    eval_strategy='no',
    save_strategy='no',
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
)
adv_trainer = Trainer(model=adv_model, args=adv_training_args, train_dataset=hf_aug, tokenizer=tokenizer)
adv_trainer.train()
adv_model.save_pretrained('./adv_mixed_model')
tokenizer.save_pretrained('./adv_mixed_model')
print("Adv-trained model saved to ./adv_mixed_model")


collected adv examples total: 391
Augmented train size: 2391


Map:   0%|          | 0/2391 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Tang Heqiang\AppData\Local\Temp\ipykernel_26948\2751664193.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  adv_trainer = Trainer(model=adv_model, args=adv_training_args, train_dataset=hf_aug, tokenizer=tokenizer)
D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
500,0.345600


Adv-trained model saved to ./adv_mixed_model


In [12]:
# Cell 10: evaluate adv-trained model on adversarial examples
adv_defense_results = {}
for name, examples in attack_outputs.items():
    texts = [e['pert_text'] for e in examples]
    labels = [e['label'] for e in examples]
    if len(texts)==0:
        adv_defense_results[name] = (None, None)
        continue
    acc, f1, preds = evaluate_texts(adv_model, tokenizer, texts, labels, batch_size=64)
    adv_defense_results[name] = (acc, f1)
    print(f"Adv-trained model on {name} adversarial examples -> acc:{acc:.4f}, f1:{f1:.4f}")


D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Adv-trained model on TextFooler adversarial examples -> acc:0.9650, f1:0.9648


D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Adv-trained model on DeepWordBug adversarial examples -> acc:0.9550, f1:0.9542


D:\Tang Heqiang\Documents\Assignment\PythonProject\.venv\Lib\site-packages\torch\nn\modules\module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Adv-trained model on BAE adversarial examples -> acc:0.9400, f1:0.9407


In [13]:
# Cell 11: summarize and save results
rows = []
for name, examples in attack_outputs.items():
    labels = [e['label'] for e in examples]
    clean_preds = [e['pred_orig'] for e in examples]
    adv_preds = [e['pred_pert'] for e in examples]
    clean_f1_local = f1_score(labels, clean_preds, average='weighted') if len(labels)>0 else None
    adv_f1_local = f1_score(labels, adv_preds, average='weighted') if len(labels)>0 else None
    spell_f1 = spell_results.get(name, None)
    advtrain_acc, advtrain_f1 = adv_defense_results.get(name, (None, None))
    rows.append({
        'attack': name,
        'clean_f1': clean_f1_local,
        'adv_f1': adv_f1_local,
        'f1_drop': (clean_f1_local-adv_f1_local) if (clean_f1_local is not None and adv_f1_local is not None) else None,
        'spell_correct_f1': spell_f1,
        'adv_train_f1': advtrain_f1
    })

summary_df = pd.DataFrame(rows)
display(summary_df)
summary_df.to_csv('mixed_adversarial_defense_summary.csv', index=False)
print("Saved mixed_adversarial_defense_summary.csv")


,attack,clean_f1,adv_f1,f1_drop,spell_correct_f1,adv_train_f1
0,TextFooler,0.985083,0.319355,0.665728,0.651351,0.964799
1,DeepWordBug,0.985083,0.347867,0.637216,0.898831,0.954195
2,BAE,0.985083,0.550991,0.434092,0.615767,0.940650


Saved mixed_adversarial_defense_summary.csv


In [14]:
print(summary_df)

        attack  clean_f1    adv_f1   f1_drop  spell_correct_f1  adv_train_f1
0   TextFooler  0.985083  0.319355  0.665728          0.651351      0.964799
1  DeepWordBug  0.985083  0.347867  0.637216          0.898831      0.954195
2          BAE  0.985083  0.550991  0.434092          0.615767      0.940650
